# RAG Using Azure AI Agent Service & Semantic Kernel

This code snippet demonstrates how to create and manage an Azure AI agent for retrieval-augmented generation (RAG) using the `Azure AI Agent Service` and `Semantic Kernel`. The agent processes user queries based on the retrieved context and provides accurate responses accordingly.

## Initializing the Environment

SQLite Version Fix
If you encounter the error:
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```

Uncomment this code block at the start of your notebook:

In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importing Packages
The following code imports the necessary packages:

In [3]:
import azure.ai.projects
print(azure.ai.projects.__version__)

1.0.0b11


In [4]:
pip install --upgrade azure-ai-projects

  Attempting uninstall: azure-ai-projects
    Found existing installation: azure-ai-projects 1.0.0b11
    Uninstalling azure-ai-projects-1.0.0b11:
      Successfully uninstalled azure-ai-projects-1.0.0b11
Note: you may need to restart the kernel to use updated packages.


In [1]:
import azure.ai.projects
print(azure.ai.projects.__version__)

1.0.0b12


In [4]:
import azure.ai.projects.models
print(dir(azure.ai.projects.models))

['AgentEvaluation', 'AgentEvaluationRedactionConfiguration', 'AgentEvaluationRequest', 'AgentEvaluationResult', 'AgentEvaluationSamplingConfiguration', 'ApiKeyCredentials', 'AssetCredentialResponse', 'AttackStrategy', 'AzureAISearchIndex', 'AzureOpenAIModelConfiguration', 'BaseCredentials', 'BlobReference', 'Connection', 'ConnectionType', 'CosmosDBIndex', 'CredentialType', 'CustomCredential', 'DatasetType', 'DatasetVersion', 'Deployment', 'DeploymentType', 'EmbeddingConfiguration', 'EntraIDCredentials', 'Evaluation', 'EvaluatorConfiguration', 'EvaluatorIds', 'FieldMapping', 'FileDatasetVersion', 'FolderDatasetVersion', 'Index', 'IndexType', 'InputData', 'InputDataset', 'ManagedAzureAISearchIndex', 'ModelDeployment', 'NoAuthenticationCredentials', 'PendingUploadRequest', 'PendingUploadResponse', 'PendingUploadType', 'RedTeam', 'RiskCategory', 'SASCredentials', 'SasCredential', 'Sku', 'TYPE_CHECKING', 'TargetConfig', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__load

In [8]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileDatasetVersion, AzureAISearchIndex, AzureOpenAIModelConfiguration
from azure.identity.aio import DefaultAzureCredential
import os

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Retrieval-Augmented Generation with Semantic Kernel & Azure AI Agent Service

This sample demonstrates how to use the **Azure AI Agent Service** to perform **Retrieval-Augmented Generation (RAG)** by combining a language model with domain-specific context from an uploaded document.

### How It Works

1. **Document Upload**: A markdown file (document.md) containing information (Contoso's travel insurance policy) is uploaded to the agent service.

2. **Vector Store Creation**: The document is indexed into a vector store to enable semantic search over its contents.

3. **Agent Configuration**: An agent is instantiated using the `gpt-4o` model with the following strict instructions:
   - Only answer questions based on retrieved content from the document.
   - Decline to answer if the question is out of scope.

4. **File Search Tool Integration**: The `FileSearchTool` is registered with the agent, enabling the model to search and retrieve relevant snippets from the indexed document during inference.

5. **User Interaction**: Users can ask questions. If relevant information is found in the document, the agent generates a grounded answer.  
   If not, the agent explicitly responds that the document does not contain sufficient information.


### Main Function


Make sure to first run `az login` using the Azure CLI so that the proper authentication context is provided while using the `DefaultAzureCredential`. The Azure AI Agent Service does not use API keys.

In [9]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds,endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT")) as client,
    ):
        file: FileDatasetVersion = await client.agents.files.upload_and_poll(file_path="document.md", purpose="assistants")
        vector_store: AzureAISearchIndex = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileDatasetVersion(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable. The requested identity has not been assigned to this resource. Error: Unexpected response "{'error': 'invalid_request', 'error_description': 'Identity not found'}"
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: ERROR: AADSTS70043: The refresh token has expired or is invalid due to sign-in frequency checks by conditional access. The token was issued on 2025-07-18T03:03:45.4791579Z and the maximum allowed lifetime for this request is 43200. Trace ID: 98c9c620-c92e-4c82-bef9-ac5199332c00 C

ClientAuthenticationError: DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable. The requested identity has not been assigned to this resource. Error: Unexpected response "{'error': 'invalid_request', 'error_description': 'Identity not found'}"
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: ERROR: AADSTS70043: The refresh token has expired or is invalid due to sign-in frequency checks by conditional access. The token was issued on 2025-07-18T03:03:45.4791579Z and the maximum allowed lifetime for this request is 43200. Trace ID: 98c9c620-c9

You should see output similar to:

```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```